<a href="https://colab.research.google.com/github/DS769/RDWH/blob/main/1DWMySQL_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datawarehousing with MySQL - StartUP

based on the codes available with the book Dimensional Datawarehousing 
available here https://drive.google.com/file/d/1Sx6f1y2mXTswCllmyVgj_AsM9jVbOMnU/view?usp=sharing


# Install MySQL

In [ ]:
!apt -y install mysql-server > /dev/null
!/etc/init.d/mysql restart
!mysql -e 'show databases'



 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


#Get Code & Data Files

In [ ]:
#!wget https://brainysoftware.com/source/9780975212820.zip
# archive available here https://drive.google.com/file/d/1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0/view?usp=sharing
#!gdown https://drive.google.com/uc?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0
!wget -q -O dwCode.zip https://github.com/Praxis-QR/RDWH/blob/main/dwMySQL/dwCode.zip?raw=true

In [ ]:
#!unzip 9780975212820.zip
!unzip dwCode.zip

Archive:  dwCode.zip
   creating: scripts/
  inflating: scripts/across_all_dimensions.sql  
  inflating: scripts/across_date.sql  
  inflating: scripts/across_date_customer.sql  
  inflating: scripts/across_date_product.sql  
  inflating: scripts/additive_data.sql  
  inflating: scripts/add_campaign_session.sql  
  inflating: scripts/add_four_milestones.sql  
  inflating: scripts/add_sales_order_quantity.sql  
  inflating: scripts/add_two_sales_orders.sql  
  inflating: scripts/annual_aggregation.sql  
  inflating: scripts/annual_tables.sql  
  inflating: scripts/balance_across_months.sql  
  inflating: scripts/balance_across_products.sql  
  inflating: scripts/band_dim.sql    
  inflating: scripts/band_ini.sql    
  inflating: scripts/band_regular.sql  
  inflating: scripts/campaign_session.sql  
  inflating: scripts/campaign_session_20.sql  
  inflating: scripts/campaign_session_21.sql  
  inflating: scripts/campaign_session_path.sql  
  inflating: scripts/campaign_stg_20.sql  
  inf

#Part 1 - Fundamentals

## Chapter 1 Initial Setup

In [ ]:
#Create User ID
#!cat scripts/create_user_id.sql
!mysql < scripts/create_user_id.sql

In [ ]:
#!cat scripts/create_databases.sql
!mysql -udwid -ppw < scripts/create_databases.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw -e "show databases like 'dw'"

mysql: [Warning] Using a password on the command line interface can be insecure.
+---------------+
| Database (dw) |
+---------------+
| dw            |
+---------------+


In [ ]:
#!cat scripts/create_dw_tables.sql
!mysql -udwid -ppw < scripts/create_dw_tables.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show tables"

+------------------+
| Tables_in_dw     |
+------------------+
| customer_dim     |
| date_dim         |
| order_dim        |
| product_dim      |
| sales_order_fact |
+------------------+


In [ ]:
!mysql -udwid -ppw < scripts/customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/more_customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2022-09-28     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2022-09-28     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2022-09-28     | 9999-12-31  |
|           4 |       

In [ ]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-01' "

In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2022-09-28     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2022-09-28     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2022-09-28     | 9999-12-31  |
|           4 |       

Creating Staging Table & Applying SCD1

In [ ]:
#!mysql -udwid -ppw < scripts/create_customer_stg.sql #Load command does not work in colab
!mysql dw -e "DROP IF TABLE EXISTS customer_stg"
!mysql dw -e "CREATE TABLE customer_stg (customer_number INT, customer_name CHAR (30) ,customer_street_address CHAR (30), customer_zip_code INT (5), customer_city CHAR (30),customer_state CHAR (2))"
!mysql dw -e "INSERT INTO customer_stg (customer_number, customer_name, customer_street_address,customer_zip_code,customer_city,country,customer_state) VALUES (1,'Really Large Customers', '7500 Louise Dr.', '17050', 'Mechanicsburg',' PA'),(2,'Small Stores','2500 Woodland St.','17055','Pittsburgh','PA'),(3,'Medium Retailers','1111 Ritter Rd'.,'17055','Pittsburgh','PA'),(4,'Good Companies',' 9500 Scott St.','17050', 'Mechanicsburg','PA'),(5,'Wonderful Shops', '3333 Rossmoyne Rd.', '17050', 'Mechanicsburg', 'PA'),(6,'Loyal Clients', '7070 Ritter Rd.', '17055', 'Pittsburgh, PA'),(7,'Distinguished Partners','9999 Scott St.', '17050', 'Mechanicsburg', 'PA')"



ERROR 1064 (42000) at line 1: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'IF TABLE EXISTS customer_stg' at line 1
ERROR 1050 (42S01) at line 1: Table 'customer_stg' already exists
ERROR 1064 (42000) at line 1: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '.,'17055','Pittsburgh','PA'),(4,'Good Companies',' 9500 Scott St.','17050', 'Mec' at line 1


In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-02' "

In [ ]:
!mysql -udwid -ppw < scripts/scd1.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-02     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-02     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-02     | 9999-12-31  |
|           4 |       

In [ ]:
!mysql -udwid -ppw < scripts/create_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/load_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 17: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-03' "


In [ ]:
!mysql -udwid -ppw < scripts/scd2.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "Select * from product_dim"

In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-05' "

In [ ]:
!mysql -udwid -ppw < scripts/load_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 17: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql -udwid -ppw < scripts/scd2.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "Select * from product_dim"

In [ ]:
!mysql -udwid -ppw < scripts/additive_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/across_all_dimensions.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
sum_of_order_amount
58000.00


In [ ]:
!mysql -udwid -ppw < scripts/across_date_product.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_number	sum_of_order_amount
1	7000.00
2	9000.00
3	12000.00
4	14000.00
5	16000.00


In [ ]:
#!mysql -udwid -ppw < scripts/across_date_customer_order.sql - file not present in the zipped folder
!mysql dw -e "SELECT product_code,SUM(order_amount) sum_of_order_amount FROM sales_order_fact a, product_dim b WHERE a.product_sk = b.product_sk GROUP BY product_code"


In [ ]:
!mysql -udwid -ppw < scripts/across_date_customer.sql #file not present in the zipped folder

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/across_date.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-06' "

In [ ]:
!mysql -udwid -ppw < scripts/dimensional_query_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e " USE dw ,SELECT date,SUM (order_amount),COUNT(*) FROM ,sales_order_fact a ,date_dim b WHERE a.order_date_sk = b.date_sk GROUP BY date ORDER BY date"
#daily_aggregation error #databse is not selected in the code within the file & editing it in the console with USE dw does not give output

In [ ]:
!mysql -udwid -ppw < scripts/daily_aggregation.sql #databse is not selected in the code within the file & editing it in the console with USE dw does not give output

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1046 (3D000) at line 7: No database selected


In [ ]:
!mysql -udwid -ppw < scripts/annual_aggregation.sql #databse is not selected in the code within the file & editing it in the console with USE dw does not give output

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1046 (3D000) at line 7: No database selected


In [ ]:
!mysql -udwid -ppw < scripts/monthly_storage.sql #no output in colab console

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/quarterly_mechanicsburg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_city	quarter	year	SUM(order_amount)	COUNT(order_sk)
Mechanicsburg	4	2005	177000.00	10


In [ ]:
!mysql -udwid -ppw < scripts/monthly_product_performer.sql
#Group by has month_name & Order by has month

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1055 (42000) at line 9: Expression #1 of ORDER BY clause is not in GROUP BY clause and contains nonaggregated column 'dw.c.month' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by


In [ ]:
!mysql dw -e "USE dw SELECT month_name, year, product_name, SUM (order_amount), COUNT(*) FROM sales_order_fact a, product_dim b, date_dim c WHERE a.product_sk = b.product_sk] AND a.order_date_sk = c.date_sk GROUP BY month_name, year, product_name HAVING SUM (order_amount) >= 75000 ORDER BY month_name , year, product_name"
#Have corrected the month_name in Order by but no output


In [ ]:
!mysql -udwid -ppw < scripts/loyal_customer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_number	year	COUNT(*)
1	2005	4


In [ ]:
!mysql -udwid -ppw < scripts/push_sales_order.sql #Push by source CDC on Sales Orders Extraction

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show procedure status like 'push_sales_order'"

+--------+------------------+-----------+----------------+---------------------+---------------------+---------------+---------+----------------------+----------------------+--------------------+
| Db     | Name             | Type      | Definer        | Modified            | Created             | Security_type | Comment | character_set_client | collation_connection | Database Collation |
+--------+------------------+-----------+----------------+---------------------+---------------------+---------------+---------+----------------------+----------------------+--------------------+
| source | push_sales_order | PROCEDURE | dwid@localhost | 2022-09-28 18:15:30 | 2022-09-28 18:15:30 | DEFINER       |         | utf8                 | utf8_general_ci      | latin1_swedish_ci  |
+--------+------------------+-----------+----------------+---------------------+---------------------+---------------+---------+----------------------+----------------------+--------------------+


In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-07' " 

ERROR 1146 (42S02) at line 1: Table 'source.customer_dim' doesn't exist


In [ ]:
!mysql source -e "use source" #using source database

In [ ]:
!mysql source -e "call push_sales_order()" #calling push_sales_order procedure

In [ ]:
!mysql dw -e "use dw" #changing datbase to dw

In [ ]:
!mysql dw -e "select* from sales_order_fact"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
|       11 |           1 |          2 |             2 |     20000.00 |
|     

In [ ]:
!mysql -udwid -ppw < scripts/create_sales_order.sql #Testing

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1050 (42S01) at line 9: Table 'sales_order' already exists


In [ ]:
!mysql -udwid -ppw < scripts/push_data.sql #Testing

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
#Stored procedure to pre-populate the date dimension
!mysql -udwid -ppw < scripts/pre_populate_date.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show procedure status like 'pre_populate_date'"

+----+-------------------+-----------+----------------+---------------------+---------------------+---------------+---------+----------------------+----------------------+--------------------+
| Db | Name              | Type      | Definer        | Modified            | Created             | Security_type | Comment | character_set_client | collation_connection | Database Collation |
+----+-------------------+-----------+----------------+---------------------+---------------------+---------------+---------+----------------------+----------------------+--------------------+
| dw | pre_populate_date | PROCEDURE | dwid@localhost | 2022-09-28 18:30:13 | 2022-09-28 18:30:13 | DEFINER       |         | utf8                 | utf8_general_ci      | latin1_swedish_ci  |
+----+-------------------+-----------+----------------+---------------------+---------------------+---------------+---------+----------------------+----------------------+--------------------+


In [ ]:
!mysql dw -e "truncate date_dim"

In [ ]:
!mysql dw - e "call pre_populate_date ('2007-01-01', '2010-12-31')"

mysql  Ver 14.14 Distrib 5.7.39, for Linux (x86_64) using  EditLine wrapper
Copyright (c) 2000, 2022, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Usage: mysql [OPTIONS] [database]
  -?, --help          Display this help and exit.
  -I, --help          Synonym for -?
  --auto-rehash       Enable automatic rehashing. One doesn't need to use
                      'rehash' to get table and field completion, but startup
                      and reconnecting may take a longer time. Disable with
                      --disable-auto-rehash.
                      (Defaults to on; use --skip-auto-rehash to disable.)
  -A, --no-auto-rehash 
                      No automatic rehashing. One has to use 'rehash' to get
                      table and field completion. This gives a quicker start of
                      mysql and disables rehashing on reconnect.
  --auto-vertic

In [ ]:
!mysql dw -e "select count(0) from date_dim" #the prepopulate procedure doesn't insert the 1461 rows as suggested

+----------+
| count(0) |
+----------+
|        0 |
+----------+


In [ ]:
!mysql dw - e "select * from date_dim limit 10 " #same error as before

mysql  Ver 14.14 Distrib 5.7.39, for Linux (x86_64) using  EditLine wrapper
Copyright (c) 2000, 2022, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Usage: mysql [OPTIONS] [database]
  -?, --help          Display this help and exit.
  -I, --help          Synonym for -?
  --auto-rehash       Enable automatic rehashing. One doesn't need to use
                      'rehash' to get table and field completion, but startup
                      and reconnecting may take a longer time. Disable with
                      --disable-auto-rehash.
                      (Defaults to on; use --skip-auto-rehash to disable.)
  -A, --no-auto-rehash 
                      No automatic rehashing. One has to use 'rehash' to get
                      table and field completion. This gives a quicker start of
                      mysql and disables rehashing on reconnect.
  --auto-vertic

In [ ]:
!mysql dw -e "truncate date_dim" #clearing the date_dim table

In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-07' " 

In [ ]:
#ONe-Date-Every-Day
!mysql -udwid -ppw < scripts/daily_date.sql #date 0000-00-00 gives error

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1292 (22007) at line 9: Incorrect date value: '0000-00-00' for column 'effective_date' at row 1


In [ ]:
!mysql dw - e "select * from date_dim"

mysql  Ver 14.14 Distrib 5.7.39, for Linux (x86_64) using  EditLine wrapper
Copyright (c) 2000, 2022, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Usage: mysql [OPTIONS] [database]
  -?, --help          Display this help and exit.
  -I, --help          Synonym for -?
  --auto-rehash       Enable automatic rehashing. One doesn't need to use
                      'rehash' to get table and field completion, but startup
                      and reconnecting may take a longer time. Disable with
                      --disable-auto-rehash.
                      (Defaults to on; use --skip-auto-rehash to disable.)
  -A, --no-auto-rehash 
                      No automatic rehashing. One has to use 'rehash' to get
                      table and field completion. This gives a quicker start of
                      mysql and disables rehashing on reconnect.
  --auto-vertic

In [ ]:
!mysql -udwid -ppw < scripts/source_date.sql #error for date value 0000-00-00

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1292 (22007) at line 9: Incorrect date value: '0000-00-00' for column 'effective_date' at row 9


In [ ]:
!mysql dw - e "select * from source.sales_order" #same error as before,no output 

mysql  Ver 14.14 Distrib 5.7.39, for Linux (x86_64) using  EditLine wrapper
Copyright (c) 2000, 2022, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Usage: mysql [OPTIONS] [database]
  -?, --help          Display this help and exit.
  -I, --help          Synonym for -?
  --auto-rehash       Enable automatic rehashing. One doesn't need to use
                      'rehash' to get table and field completion, but startup
                      and reconnecting may take a longer time. Disable with
                      --disable-auto-rehash.
                      (Defaults to on; use --skip-auto-rehash to disable.)
  -A, --no-auto-rehash 
                      No automatic rehashing. One has to use 'rehash' to get
                      table and field completion. This gives a quicker start of
                      mysql and disables rehashing on reconnect.
  --auto-vertic

In [ ]:
#Adding more dates from additional sales orders
!mysql -udwid -ppw < scripts/more_sales_order.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/source_date.sql #same error for date 0000-00-00

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1292 (22007) at line 9: Incorrect date value: '0000-00-00' for column 'effective_date' at row 13


In [ ]:
!mysql dw -e"select * from date_dim" #no output

In [ ]:
#initial population script
!mysql -udwid -ppw < scripts/dw_initial.sql #LOAD command does not work 

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 9: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
#Running the initial Population script
!mysql -udwid -ppw < scripts/truncate_tables.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
#Pre-Popuating the Date Dimension
!mysql dw -e "use dw"

In [ ]:
!mysql dw -e "call pre_populate_date('2005-03-01', '2010-12-31')" #0000-00-00 gives an error

ERROR 1292 (22007) at line 1: Incorrect date value: '0000-00-00' for column 'effective_date' at row 1


In [ ]:
!mysql -udwid -ppw < scripts/sales_order_initial.sql #Inserting sales into sales_order table

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-02-28' " 

In [ ]:
!mysql -udwid -ppw < scripts/dw_initial.sql #Load command

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 9: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql -udwid -ppw < scripts/confirm_initial_population.sql #syntax error in the file

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1064 (42000) at line 9: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'on
, customer_name
, product_name
, date
, order_amount amount
FROM 
  sales_ord' at line 2


In [ ]:
#Regular POpulation
!mysql -udwid -ppw < scripts/dw_regular.sql #LOAD command won't work

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 14: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql -udwid -ppw < scripts/sales_order_regular.sql #adding sales orders

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "update customer_dim set effective_date = '2007-03-01' " 

In [ ]:
!mysql -udwid -ppw < scripts/dw_regular.sql #LOad command doesn't work

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 14: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql dw -e"select * from customer_dim" #no output becuase Load function could not laod the CSV file into the table

In [ ]:
!mysql dw -e"select * from product_dim" #no output becuase Load function could not laod the CSV file into the table

In [ ]:
!mysql dw - e"select * from order_dim"

mysql  Ver 14.14 Distrib 5.7.39, for Linux (x86_64) using  EditLine wrapper
Copyright (c) 2000, 2022, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Usage: mysql [OPTIONS] [database]
  -?, --help          Display this help and exit.
  -I, --help          Synonym for -?
  --auto-rehash       Enable automatic rehashing. One doesn't need to use
                      'rehash' to get table and field completion, but startup
                      and reconnecting may take a longer time. Disable with
                      --disable-auto-rehash.
                      (Defaults to on; use --skip-auto-rehash to disable.)
  -A, --no-auto-rehash 
                      No automatic rehashing. One has to use 'rehash' to get
                      table and field completion. This gives a quicker start of
                      mysql and disables rehashing on reconnect.
  --auto-vertic

In [ ]:
!mysql dw -e "select* from sales_order_fact"

In [ ]:
#Regular Population scheduling
!mysql -udwid -ppw < scripts\dw_regular.sql # no file path found error

/bin/bash: scriptsdw_regular.sql: No such file or directory


Last step is scheduling the batch job via Schedule Task wizard in control panel
